<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RNN Multivariate Forecasting**

This tutorial is an introduction to time series forecasting using Recurrent Neural Networks (RNNs). This is covered in two parts: first, you will forecast a univariate time series, then you will forecast a multivariate time series.

**Import Libraries**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## Import Dataset
This tutorial uses a <a href="https://www.bgc-jena.mpg.de/wetter/" class="external">[weather time series dataset</a> recorded by the <a href="https://www.bgc-jena.mpg.de" class="external">Max Planck Institute for Biogeochemistry</a>.

This dataset contains 14 different features such as air temperature, atmospheric pressure, and humidity. These were collected every 10 minutes, beginning in 2003. For efficiency, you will use only the data collected between 2009 and 2016. This section of the dataset was prepared by François Chollet for his book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python).

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [ ]:
df = pd.read_csv(csv_path)

Let's take a glance at the data.

In [ ]:
df.head()

As you can see above, an observation is recorded every 10 mintues. This means that, for a single hour, you will have 6 observations. Similarly, a single day will contain 144 (6x24) observations. 

Given a specific time, let's say you want to predict the temperature 6 hours in the future. In order to make this prediction, you choose to use 5 days of observations. Thus, you would create a window containing the last 720(5x144) observations to train the model. Many such configurations are possible, making this dataset a good one to experiment with.

The function below returns the above described windows of time for the model to train on. The parameter `history_size` is the size of the past window of information. The `target_size` is how far in the future does the model need to learn to predict. The `target_size` is the label that needs to be predicted.

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

In both the following tutorials, the first 300,000 rows of the data will be the training dataset, and there remaining will be the validation dataset. This amounts to ~2100 days worth of training data.

In [ ]:
TRAIN_SPLIT = 300000

Setting seed to ensure reproducibility.

In [ ]:
tf.random.set_seed(13)

## Forecast a multivariate time series

The original dataset contains fourteen features. For simplicity, this section considers only three of the original fourteen. The features used are air temperature, atmospheric pressure, and air density. 

To use more features, add their names to this list.

In [ ]:
features_considered = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']

In [ ]:
features = df[features_considered]
features.index = df['Date Time']
features.head()

Let's have a look at how each of these features vary across time.

In [ ]:
features.plot(subplots=True)

As mentioned, the first step will be to standardize the dataset using the mean and standard deviation of the training data.

In [ ]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)

In [ ]:
dataset = (dataset-data_mean)/data_std

#### Single step model
In a single step setup, the model learns to predict a single point in the future based on some history provided.

The below function performs the same windowing task as below, however, here it samples the past observation based on the step size given.

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In this tutorial, the network is shown data from the last five (5) days, i.e. 720 observations that are sampled every hour. The sampling is done every one hour since a drastic change is not expected within 60 minutes. Thus, 120 observation represent history of the last five days.  For the single step prediction model, the label for a datapoint is the temperature 12 hours into the future. In order to create a label for this, the temperature after 72(12*6) observations is used.

In [ ]:
past_history = 720
future_target = 72
STEP = 6

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

Let's look at a single data-point.


In [ ]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [ ]:
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [ ]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

Let's check out a sample prediction.

In [ ]:
for x, y in val_data_single.take(1):
  print(single_step_model.predict(x).shape)

In [ ]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)

In [ ]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [ ]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

##### Predict a single step future
Now that the model is trained, let's make a few sample predictions. The model is given the history of three features over the past five days sampled every hour (120 data-points), since the goal is to predict the temperature, the plot only displays the past temperature. The prediction is made one day into the future (hence the gap between the history and prediction). 

In [ ]:
for x, y in val_data_single.take(3):
  plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 12,
                   'Single Step Prediction')
  plot.show()

#### Multi-Step model
In a multi-step prediction model, given a past history, the model needs to learn to predict a range of future values. Thus, unlike a single step model, where only a single future point is predicted, a multi-step model predict a sequence of the future.

For the multi-step model, the training data again consists of recordings over the past five days sampled every hour. However, here, the model needs to learn to predict the temperature for the next 12 hours. Since an obversation is taken every 10 minutes, the output is 72 predictions. For this task, the dataset needs to be prepared accordingly, thus the first step is just to create it again, but with a different target window.

In [ ]:
future_target = 72
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

Let's check out a sample data-point.

In [ ]:
print ('Single window of past history : {}'.format(x_train_multi[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train_multi[0].shape))

In [ ]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

Plotting a sample data-point.

In [ ]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

In this plot and subsequent similar plots, the history and the future data are sampled every hour.

In [ ]:
for x, y in train_data_multi.take(1):
  multi_step_plot(x[0], y[0], np.array([0]))

Since the task here is a bit more complicated than the previous task, the model now consists of two LSTM layers. Finally, since 72 predictions are made, the dense layer outputs 72 predictions.

In [ ]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,
                                          input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(72))

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')

Let's see how the model predicts before it trains.

In [ ]:
for x, y in val_data_multi.take(1):
  print (multi_step_model.predict(x).shape)

In [ ]:
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

In [ ]:
plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')

##### Predict a multi-step future
Let's now have a look at how well your network has learnt to predict the future.

In [ ]:
for x, y in val_data_multi.take(3):
  multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

## Next steps
This tutorial was a quick introduction to time series forecasting using an RNN. You may now try to predict the stock market and become a billionaire.

In addition, you may also write a generator to yield data (instead of the uni/multivariate_data function), which would be more memory efficient. You may also check out this [time series windowing](https://www.tensorflow.org/guide/data#time_series_windowing) guide and use it in this tutorial.

For further understanding, you may read Chapter 15 of [Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/), 2nd Edition and Chapter 6 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python).

# **LSTM Univariate Forecasting**

https://machinelearningmastery.com/time-series-prediction-with-deep-learning-in-python-with-keras/

https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import LSTM, Dense

print('Tensorflow version %s' % tf.__version__)

!pip install --quiet ipython-autotime pandas_gbq
%load_ext autotime

#### **Load Data**

**Sunspot Dataset**

https://blogs.rstudio.com/ai/posts/2018-06-25-sunspots-lstm/

In [ ]:
# https://www.kaggle.com/robervalt/sunspots
# https://www.kaggle.com/leeminwoo/keras-bidirectonal-lstm
url = 'https://raw.githubusercontent.com/deltorobarba/repo/master/sunspots.csv.xls'
data = pd.read_csv(url)
data = data[['Date', 'Monthly Mean Total Sunspot Number']].rename(columns={"Date": "date", "Monthly Mean Total Sunspot Number": "sunspots"}).set_index('date')
data.index = pd.to_datetime(data.index, utc=False)
dataset = data.copy()
dataset.head()

**Airline Passengers**

In [ ]:
#url = 'https://raw.githubusercontent.com/deltorobarba/repo/master/airline-passengers.csv'
#data = pd.read_csv(url)
#data = data[['Month', 'Passengers']].rename(columns={"Month": "date", "Passengers": "passengers"}).set_index('date')
#data.index = pd.to_datetime(data.index, utc=False)
#dataset = data.copy()
#dataset.head()

**Google Stock Data**

In [ ]:
# url = 'https://raw.githubusercontent.com/deltorobarba/repo/master/google.csv'
# data = pd.read_csv(url)
# series = data[['Date', 'Close']].rename(columns={"Date": "date", "Close": "values"}).set_index('date')
# series.index = pd.to_datetime(series.index, utc=False)
# dataset = series.copy()
# dataset.head()

Diverse stock data

In [ ]:
#url = 'https://raw.githubusercontent.com/deltorobarba/repo/master/timeseries.csv'
#data = pd.read_csv(url)
#series = data[['date', 'price1', 'price2']].rename(columns={"price1": "google", "price2": "apple"}).set_index('date')
#series.index = pd.to_datetime(series.index, utc=False)
#dataset = series.copy()
#dataset.head()

**Insolvenzverfahren**

In [ ]:
# url = 'https://raw.githubusercontent.com/deltorobarba/repo/master/sh_unmelt_bq_raw.csv'
# sh_unmelt_bq_raw = pd.read_csv(url)
# sh_unmelt_bq_raw = sh_unmelt_bq_raw.sort_values('datum')
# dataset = sh_unmelt_bq_raw.copy()
# dataset = dataset.drop(columns=['Bergbau', 'Dienstleistungen', 'Energieversorgung','Erziehung', 'Finanzleistungen', 'Gastgewerbe','IT', 'Manufaktur', 'Sonstiges','Sozialwesen', 'Verkehr', 'Wohnungswesen'])
# dataset = dataset.set_index('datum')
# dataset.index = pd.to_datetime(dataset.index, utc=False)
# dataset.head()

#### **Set input data**

In [ ]:
# Set data
data = dataset.copy()

# Set seed for reproducibility
tf.random.set_seed(7)

#### **Normalize input data**

https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)
# data = data[:-1, :] # for multivariate datasets: remove last row due to nan

In [ ]:
print(data.shape)
# print(dataset.dtype)

In [ ]:
# Display dataset
data [0:10] # Display first 10 values

#### **Train & Test Split**

In [ ]:
# Train & Test Split in % of complete dataset
split = 0.9
train_size = int(len(data) * split)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [ ]:
train [0:10] # Display first 10 values

In [ ]:
test [0:10] # Display first 10 values

Now train & test data have been separated. 

We need to divide by feature input (X) and prediction variable (Y).

For both train and test dataset

#### **Window & Reshape**

In [ ]:
window = 12 # Use (t), (t-1) and (t-2) as input variables X
features = 1 # we have only one time series

https://machinelearningmastery.com/time-series-prediction-with-deep-learning-in-python-with-keras/

In [ ]:
# Convert an array of values into a dataset matrix
def create_dataset(data, window=window):
    dataX, dataY = [], []
    for i in range(len(data)-window-1):
        a = data[i:(i+window), 0]
        dataX.append(a)
        dataY.append(data[i + window, 0])
    return np.array(dataX), np.array(dataY)

# X is input data = value (at t, and t-1, t-2, depending on window size)
# Y is target data = value at t+1
# Each is additionally divided into train and test set

In [ ]:
# Reshape into X=t and Y=t+1 and get as arrays
trainX, trainY = create_dataset(train, window)
testX, testY = create_dataset(test, window)

In [ ]:
print(trainX.shape)

In [ ]:
print(trainY.shape)

In [ ]:
# X input has 3 (= window size) values at each step
trainX [0:10] # Display first 10 values

In [ ]:
# Y output has here only one value (t+1) at each step
trainY

In [ ]:
trainX.shape[0]

In [ ]:
trainX.shape[1]

#### **Reshape Data Format for LSTM**

https://www.kaggle.com/shivajbd/input-and-output-shape-in-lstm-keras

In [ ]:
# Reshape Data Format for LSTM (simple model)

# Data is in shape **[samples, features]**. LSTM expects **[samples, time steps, features]**
# Reminder 2D array: shape (n,m), n [0] = rows (time steps), m [1] = column(s)
# Shape: First Block: t, t+1, t+2. Second: t+1, t+2, t+3. Third: t+2, t+3, t+4.
# Approach 1: Use past observations as separate input features =(trainX.shape[0], features, trainX.shape[1])

# trainX = np.reshape(trainX, (trainX.shape[0], features, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], features, testX.shape[1]))

In [ ]:
trainX

In [ ]:
# Approach 2: Use past observations as time steps of the one input feature

trainX = tf.reshape(trainX, (trainX.shape[0], trainX.shape[1], features))
testX = tf.reshape(testX, (testX.shape[0], testX.shape[1], features))
trainX

In [ ]:
print(trainX.shape)

In [ ]:
trainX [0:10] # Display first 10 values

In [ ]:
print(testX.shape)

#### **Model Training**

In [ ]:
# Set Hyperparameter
optimizer = tf.keras.optimizers.Adam(0.0001)
loss = 'mean_squared_error' #tf.keras.losses.LogCosh() or 'huber'
activation = 'relu'
dropout = 0.002
epochs = 15
verbose = 1
units = 1 # LSTM output units
repeat = 1
batch_size = 2

In [ ]:
model = keras.Sequential(name="my_sequential")

# https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

# Add a LSTM layer with internal units.
model.add(layers.LSTM(units, 
                      input_shape=(window, features),
                      dropout = dropout,
                      return_sequences=False, 
                      activation=activation, 
                      name="layer1"))

# Add a Dense layer with 1 unit (das aktivieren, falls LSTM output units > 1)
# model.add(layers.Dense(1, activation = 'relu', name="layer2"))

# Call model on a test input
trainY = model(trainX)

In [ ]:
model.summary()

In [ ]:
trainY.shape

In [ ]:
# model.weights

In [ ]:
# print("Number of weights after calling the model:", len(model.weights))

In [ ]:
model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=['mean_squared_error'])

In [ ]:
# Prepare visualisation - Plot Losses Keras TF (simple)
!pip install livelossplot --quiet
from livelossplot import PlotLossesKerasTF

# Select visualization method
callbacks = [PlotLossesKerasTF()]

In [ ]:
# Train model
model.fit(trainX,
          trainY, 
          epochs=epochs, 
          verbose=verbose, 
          shuffle=False, 
          callbacks=callbacks)

#### **Make Predictions**

In [ ]:
trainPredict = model.predict(trainX)
model.reset_states()
testPredict = model.predict(testX)

#### **Predictions (Train & Test)**

**Test Predict**

In [ ]:
testPredict [0:10]

In [ ]:
testPredict = scaler.inverse_transform(testPredict)
testPredict [0:10]

**Train Predict**

In [ ]:
trainPredict [0:10]

In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
trainPredict [0:10]

#### **Actual (Train & Test)**

**Test Actual**

In [ ]:
testY [0:10]

In [ ]:
testY = scaler.inverse_transform([testY])
testY [0:10]

**Train Actual**

In [ ]:
trainY [0:10]

In [ ]:
# Convert / Create a numpy ndarray from a tensor
trainY = tf.make_tensor_proto(trainY)  # convert `tensor a` to a proto tensor
trainY = tf.make_ndarray(trainY)
trainY [0:10]

In [ ]:
trainY.shape

In [ ]:
# trainY = scaler.inverse_transform([trainY])
trainY [0:10]

#### **Residuals**

In [ ]:
# Get residuals
residuals = (testY[0] - testPredict[:,0])
res = pd.DataFrame(data=residuals, columns=['residuals'])
res.head()

In [ ]:
# Plot Residuals
sns.set(rc={'figure.figsize':(15, 8)})
res.plot(title='Residuals')

#### **Calculate RMSE & Plot**

In [ ]:
# Inverse original data
data = scaler.inverse_transform(data)
data [0:10]

In [ ]:
# Plot results

# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[window:len(trainPredict)+window, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(window*2)+1:len(dataset)-1, :] = testPredict

# plot baseline and predictions
sns.set(rc={'figure.figsize':(11, 5)})
#plt.plot(scaler.inverse_transform(dataset))
plt.plot(data)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
# print('Train Score: %.2f RMSE' % (trainScore))

# **LSTM Multivariate Forecasting (Normalized)**

## **Import Libraries**

https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/

In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
import seaborn as sns
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from numpy import concatenate


from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import LSTM, Dense

print('Tensorflow version %s' % tf.__version__)

!pip install --quiet ipython-autotime pandas_gbq
%load_ext autotime

## **Preprocess Data**

#### **Import & Preprocess Data**

In [ ]:
from pandas import read_csv
from datetime import datetime
# load data
def parse(x):
	return datetime.strptime(x, '%Y %m %d %H')
dataset = pd.read_csv('https://raw.githubusercontent.com/deltorobarba/repo/master/pollution.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
dataset.drop('No', axis=1, inplace=True)
# manually specify column names
dataset.columns = ['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']
dataset.index.name = 'date'
# mark all NA values with 0
dataset['pollution'].fillna(0, inplace=True)
# drop the first 24 hours
dataset = dataset[24:]
# summarize first 5 rows
print(dataset.head(5))

In [ ]:
url = 'https://raw.githubusercontent.com/deltorobarba/repo/master/timeseries.csv'
data = pd.read_csv(url)
series = data[['date', 'price1', 'price2']].rename(columns={"price1": "google", "price2": "apple"}).set_index('date')
series.index = pd.to_datetime(series.index, utc=False)
dataset = series.copy()
dataset.head()

#### **Visualize Dataset**

In [ ]:
sns.set(rc={'figure.figsize':(10, 8), "lines.linewidth": 1.0})

# load dataset
values = dataset.values
# specify columns to plot
groups = [0, 1]
i = 1
# plot each column
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups), 1, i)
	pyplot.plot(values[:, group])
	pyplot.title(dataset.columns[group], y=0.5, loc='right')
	i += 1
pyplot.show()

#### **LSTM Preprocess Dataset**

In [ ]:
windows = 3
features = 2

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
#dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values

In [ ]:
# ensure all data is float
values = values.astype('float32')

In [ ]:
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [ ]:
# frame as supervised learning
reframed = series_to_supervised(scaled, windows, 1)

In [ ]:
# Display data
reframed [:5]

In [ ]:
# Show last column (to be removed)
num_rows, num_cols = reframed.shape
n = print (num_cols)
n

In [ ]:
# drop columns we don't want to predict
n = 7 # number of last column (from above) minus 1 (the index column)
reframed.drop(reframed.columns[[n]], axis=1, inplace=True)
print(reframed.head())

#### **Shape for LSTM**

In [ ]:
# how many rows (time stamps)?
reframed.shape[0]

In [ ]:
values = reframed.values
train_time = 2000
train = values[:train_time, :]
test = values[train_time:, :]

In [ ]:
# split into input and outputs
n_obs = windows * features
train_X, train_y = train[:, :n_obs], train[:, -features]
test_X, test_y = test[:, :n_obs], test[:, -features]
print(train_X.shape, len(train_X), train_y.shape)

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], windows, features))
test_X = test_X.reshape((test_X.shape[0], windows, features))

In [ ]:
print(test_X.shape, len(test_X), test_y.shape)

In [ ]:
test_X

## **Model Fitting & Forecasting**

#### **Create & Train Model**

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
# Fit model
history = model.fit(train_X, train_y, 
                    epochs=20, 
                    batch_size=72, 
                    validation_data=(test_X, test_y), 
                    verbose=1, 
                    shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

#### **Make a prediction**

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], windows*features))

In [ ]:
# Shape: 6 windows (3 timesteps for 2 features)
test_X.shape[0], test_X.shape[1]

In [ ]:
test_X [0:5]

In [ ]:
# Shape: one timestep forecasted for one feature
yhat.shape[0], yhat.shape[1]

In [ ]:
yhat [0:5]

## **Postprocess Data**

#### **Invert Scaling for Forecasted Test Data**

In [ ]:
# remove all other features to align matrix structure
inv_yhat = concatenate((yhat, test_X[:, -1:]), axis=1)
inv_yhat [0:5]

In [ ]:
inv_yhat.shape[0], inv_yhat.shape[1]

In [ ]:
inv_yhat = scaler.inverse_transform(inv_yhat)[:, [1]]
inv_yhat [0:5]

In [ ]:
inv_yhat = inv_yhat[:,0]

In [ ]:
inv_yhat [0:5]

#### **Invert Scaling for Actual Test Data**

In [ ]:
# Display target data
test_y [0:5]

In [ ]:
# Reshape target data to concatenate later
test_y = test_y.reshape((len(test_y), 1))
test_y [0:5]

In [ ]:
# Display input data
test_X [0:5]

In [ ]:
# Concatenate input & target data
### This is purely for demonstration !!

inv_y = concatenate((test_y, test_X[:, 0:]), axis=1)
inv_y [0:5]

In [ ]:
# Concatenate input & target data
inv_y = concatenate((test_y, test_X[:, -1:]), axis=1)
inv_y [0:5]

In [ ]:
# testplot

df = pd.DataFrame(data=inv_y)
sns.set(rc={'figure.figsize':(10, 5)})
df.plot(title='series')

In [ ]:
# Invert Scaling
inv_y = scaler.inverse_transform(inv_y)
inv_y [0:5]

In [ ]:
# testplot

df = pd.DataFrame(data=inv_y)
sns.set(rc={'figure.figsize':(10, 5)})
df.plot(title='series')

In [ ]:
# Reshape again (transpose) and leave target data only (first column)
inv_y = inv_y[:,0]
inv_y [0:5]

## **Evaluate Model**

#### **Reshape Data and Plot**

In [ ]:
df1 = inv_y.reshape((len(inv_y), 1))
# df1 [0:5]
df1 = pd.DataFrame(data=df1, columns=['Actual'])
df1.head()

In [ ]:
df2 = inv_yhat.reshape((len(inv_yhat), 1))
# df2 [0:5]
df2 = pd.DataFrame(data=df2, columns=['Forecast'])
df2.head()

In [ ]:
def residuals(actual, forecast):
    # returns the difference between post and pre
    return actual - forecast

df = pd.concat([df1,df2], axis=1)
df['Residuals'] = residuals(df['Actual'], df['Forecast'])
df.head()

In [ ]:
# Plot Actual vs Forecast Values
# df = pd.DataFrame(data=df)
sns.set(rc={'figure.figsize':(10, 5)})
df[['Actual', 'Forecast']].plot(title='Actual vs Forecast')

In [ ]:
# Plot Residuals
sns.set(rc={'figure.figsize':(10, 5)})
df[['Residuals']].plot(title='Residuals')

#### **Calculate RMSE**

In [ ]:
# inv_y : Test Actual Target Data
# inv_yhat : Test Forecasted Target Data
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.2f' % rmse)

#### **Residuals**

In [ ]:
# Get residuals (normalized)
residuals = (test_y[0] - yhat[:,0])
res = pd.DataFrame(data=residuals, columns=['residuals'])
res.head()

In [ ]:
# Plot Residuals
sns.set(rc={'figure.figsize':(10, 5)})
res.plot(title='Residuals')

#### **Plot Results**

In [ ]:
# Plot results

# shift train predictions for plotting
#trainPredictPlot = np.empty_like(dataset)
#trainPredictPlot[:, :] = np.nan
#trainPredictPlot[windows:len(yhat)+windows, :] = yhat

# shift test predictions for plotting
#testPredictPlot = np.empty_like(dataset)
#testPredictPlot[:, :] = np.nan
#testPredictPlot[len(test_X)+(windows*2)+1:len(dataset)-1, :] = test_X

# plot baseline and predictions
#sns.set(rc={'figure.figsize':(11, 5)})
#plt.plot(scaler.inverse_transform(dataset))
#plt.plot(dataset)
#plt.plot(trainPredictPlot)
#plt.plot(testPredictPlot)
#plt.show()

# **LSTM Multivariate Forecasting (Original)**

## **Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
import seaborn as sns
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from numpy import concatenate


from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import LSTM, Dense

print('Tensorflow version %s' % tf.__version__)

!pip install --quiet ipython-autotime pandas_gbq
%load_ext autotime

## **Preprocess Data**

#### **Import & Preprocess Data**

In [ ]:
from pandas import read_csv
from datetime import datetime
# load data
def parse(x):
	return datetime.strptime(x, '%Y %m %d %H')
dataset = pd.read_csv('https://raw.githubusercontent.com/deltorobarba/repo/master/pollution.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
dataset.drop('No', axis=1, inplace=True)
# manually specify column names
dataset.columns = ['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']
dataset.index.name = 'date'
# mark all NA values with 0
dataset['pollution'].fillna(0, inplace=True)
# drop the first 24 hours
dataset = dataset[24:]
# summarize first 5 rows
print(dataset.head(5))

In [ ]:
url = 'https://raw.githubusercontent.com/deltorobarba/repo/master/timeseries.csv'
data = pd.read_csv(url)
series = data[['date', 'price1', 'price2']].rename(columns={"price1": "google", "price2": "apple"}).set_index('date')
series.index = pd.to_datetime(series.index, utc=False)
dataset = series.copy()
dataset.head()

#### **Visualize Dataset**

In [ ]:
sns.set(rc={'figure.figsize':(10, 8), "lines.linewidth": 1.0})

# load dataset
values = dataset.values
# specify columns to plot
groups = [0, 1]
i = 1
# plot each column
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups), 1, i)
	pyplot.plot(values[:, group])
	pyplot.title(dataset.columns[group], y=0.5, loc='right')
	i += 1
pyplot.show()

#### **LSTM Preprocess Dataset**

In [ ]:
windows = 3
features = 2

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
#dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values

In [ ]:
# ensure all data is float
values = values.astype('float32')

In [ ]:
# remove this column if you normalize
scaled = values

In [ ]:
# frame as supervised learning
reframed = series_to_supervised(scaled, windows, 1)

In [ ]:
# Display data
reframed [:5]

In [ ]:
# Show last column (to be removed)
num_rows, num_cols = reframed.shape
n = print (num_cols)
n

In [ ]:
# drop columns we don't want to predict
n = 7 # number of last column (from above) minus 1 (the index column)
reframed.drop(reframed.columns[[n]], axis=1, inplace=True)
print(reframed.head())

#### **Shape for LSTM**

In [ ]:
# how many rows (time stamps)?
reframed.shape[0]

In [ ]:
values = reframed.values
train_time = 2000
train = values[:train_time, :]
test = values[train_time:, :]

In [ ]:
# split into input and outputs
n_obs = windows * features
train_X, train_y = train[:, :n_obs], train[:, -features]
test_X, test_y = test[:, :n_obs], test[:, -features]
print(train_X.shape, len(train_X), train_y.shape)

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], windows, features))
test_X = test_X.reshape((test_X.shape[0], windows, features))

In [ ]:
print(test_X.shape, len(test_X), test_y.shape)

In [ ]:
test_X

## **Model Fitting & Forecasting**

#### **Create & Train Model**

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
# Fit model
history = model.fit(train_X, train_y, 
                    epochs=60, 
                    batch_size=72, 
                    validation_data=(test_X, test_y), 
                    verbose=1, 
                    shuffle=False)

In [ ]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

#### **Make a prediction**

hier muss ein datensatz mit einem timestep in die zukunft rein

In [ ]:
# make a prediction
yhat = model.predict(test_X)

In [ ]:
# Shape: one timestep forecasted for one feature
yhat.shape[0], yhat.shape[1]

In [ ]:
yhat [0:5]

In [ ]:
test_y [0:5]

In [ ]:
test_y = test_y.reshape((len(test_y), 1))
test_y [0:5]

**Reshape Test data**

dieser teil ist nutzlos!

In [ ]:
# Shape: 2 features with each 3 timesteps in the past (windows)
test_X [0:5]

In [ ]:
test_X = test_X.reshape((test_X.shape[0], windows*features))

In [ ]:
# Shape: 6 windows (3 timesteps for 2 features)
test_X.shape[0], test_X.shape[1]

In [ ]:
test_X [0:5]

In [ ]:
# remove unnecessary data
test_X_reshape = test_X[:, -1:]
test_X_reshape [0:5]

#### **Evaluate on Unnormalized Data**

In [ ]:
# Concatenate series
#df1 = pd.DataFrame(data=test_X_reshape, columns=['Actual'])
df1 = pd.DataFrame(data=test_y, columns=['Actual'])
df2 = pd.DataFrame(data=yhat, columns=['Forecast'])

def residuals(Actual, Forecast):
    # returns the difference between post and pre
    return Actual - Forecast

df = pd.concat([df1,df2], axis=1)
df['Residuals'] = residuals(df['Actual'], df['Forecast'])
df.head()

In [ ]:
# Plot Residuals
sns.set(rc={'figure.figsize':(10, 5)})
df[['Residuals']].plot(title='Residuals')

In [ ]:
# Plot Actual vs Forecast Values
sns.set(rc={'figure.figsize':(10, 5)})
df[['Actual', 'Forecast']].plot(title='Actual vs Forecast')

In [ ]:
rmse = sqrt(mean_squared_error(test_y, yhat))
print('Test RMSE: %.2f' % rmse)